In [1]:
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
import datetime
import warnings
import time
warnings.filterwarnings('ignore')

# Back End Script 

### We are writing a python script to generate a table of insights for a particular user. This script will exist in one function, we can define functions within the function, but we want to streamline as much as possible for maximum efficiency. 

#### Insights To Generate  
- steps by sleep 
- sleep by heartrate 
- 

 _id: UUID (primary key)

owner: UUID (reference to User)

source: STRING - Describes where this insight came from. For example, insights created manually from the admin dashboard will have their source set as admin.

category: STRING - indicates which category/vertical the insight belongs to

type: STRING - A short identifier unique for each type of insight. Example: An insight recommending the user to take x steps to improve sleep might be identified by a type such as "step-count:sleep:achieve-steps".

message: TEXT - A textual representation of the insight, to be used in places where we can't display rich media, such as notifications. Example: "Get 2500 steps in by noon for a more restful sleep".

data: JSON - a JSON blob with detailed info on the insight. We make this a JSON field so that it can be easily extended in the future with new types of insights, without requiring changes to the DB schema.

upvote: BOOLEAN - A boolean field that is set to true when the user marks this insight as "useful" or gives it a "thumbs up". Default to false.

downvote: BOOLEAN - A boolean field that is set to true when the user marks this insight as "not useful" or gives it a "thumbs down". Default to false.


For the initial version, the JSON structure of the data field will be relatively simple:

{
    "message": "Get 2500 steps in by noon for a more restful sleep",
    "metrics": ["step-count", "sleep"],
    "explanation": "Science shows that getting physical activity and sunlight in the morning can help to keep your circadian rhythm in optimal shape. Your circadian rhythm is..."
}

In [26]:
#This is what the table is going to look like. 
#There are nine categories. Above is a short description of each column. 
UserInsight = pd.DataFrame(columns=['_id','owner','source','category','type','message','data','upvote','downvote'])
UserInsight

,_id,owner,source,category,type,message,data,upvote,downvote


In [27]:
#Example of how to insert into the table. 

blob = {"message":"If you keep losing weight","metrics":'The correlated variables',"explanation":'Leave this'}
example = ['000','John','Apple Watch','HeartRate','health-weight:heart-rate:positive','If you keep losing weight, your heart rate will drop',blob,'N/A','N/A']

UserInsight.loc[len(UserInsight)] = example
UserInsight

,_id,owner,source,category,type,message,data,upvote,downvote
0,000,John,Apple Watch,HeartRate,health-weight:heart-rate:positive,"If you keep losing weight, your heart rate wil...","{'message': 'If you keep losing weight', 'expl...",N/A,N/A


### Function  

In [32]:
#We need to tweak our functions to create an array like example above ^ 

def UserInsightFunction(user_id,data_table):
    UserInsight = pd.DataFrame(columns=['_id','owner','source','category','type','message','data','upvote','downvote'])
    InsightTypes = []
    
    def CorrelationTable(): 
        #create table of all correlations in our data set for the user.
        CorrelationTable = pd.DataFrame(columns=['Variable1','Variable2','Correlation Value','Last 30 Days','Last Week','Last 24 Hours'])
        return
    
    def InsightCreator():
        #This is where we want to create the insight arrays from the correlation table, and append to UserInsights
        return
    
    
    
    return UserInsight